In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter
from mayavi import mlab
from numba import jit

# ===================================
# Step 1: Create Smooth Terrain
# ===================================
grid_size = 5
resolution = 0.05  # Slightly coarser resolution for better performance
x = np.arange(0, grid_size, resolution)
y = np.arange(0, grid_size, resolution)
X, Y = np.meshgrid(x, y)

amplitude_variation = 0.3
frequency_variation_x = 2 * np.pi * (1 * np.random.rand(*X.shape))
frequency_variation_y = 2 * np.pi * (1 * np.random.rand(*Y.shape))
Z = amplitude_variation * np.sin(frequency_variation_x * X) * np.cos(frequency_variation_y * Y)

num_bumps = 2000
for _ in range(num_bumps):
    x_center = np.random.uniform(0, 0)
    y_center = np.random.uniform(0, 0)
    height = 0.025 * np.random.rand()
    width = 0.05 + 0.1 * np.random.rand()
    Z += height * np.exp(-((X - x_center)**2 + (Y - y_center)**2) / (2 * width**2))

Z = np.abs(Z)
Z = gaussian_filter(Z, sigma=20)


In [ ]:
# ===================================
# Step 2: Particle Initialization
# ===================================
particle_radius = 0.01
particle_diameter = 2 * particle_radius


positions = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        # Terrain height at this grid point
        terrain_height = Z[i, j]
        # Number of particles to stack at this grid point
        num_particles = int(terrain_height / particle_diameter)
        # Stack particles vertically
        for k in range(num_particles):
            px, py = X[i, j], Y[i, j]
            pz = k * particle_diameter + particle_radius  # Center of the particle
            positions.append([px, py, pz])

positions = np.array(positions)

velocities = np.zeros_like(positions)
masses = np.ones(len(positions))
radii = np.full(len(positions), particle_radius)


# ===================================
# Step 3: Visualization with Mayavi
# ===================================
mlab.figure(size=(800, 600), bgcolor=(0.9, 0.9, 1))

# Plot terrain as a surface
mlab.surf(X, Y, Z.T, colormap='terrain', warp_scale=1)

# Plot particles
particle_plot = mlab.points3d(
    positions[:, 0], positions[:, 1], positions[:, 2],
    scale_factor=particle_diameter,
    color=(0.8, 0.7, 0.6),
)

# Draw rover as a cuboid
rover_center = [2.5, 2.5, 0.2]   # Rover's initial position
rover_size = [0.4, 0.4, 0.2]              # Rover's dimensions (length x width x height)
draw_solid_cuboid(rover_center, rover_size)

# Add axes and set view
mlab.axes(extent=[0, grid_size, 0, grid_size, 0, Z.max()])
mlab.view(azimuth=45, elevation=60, distance=8)

mlab.show()


In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from mayavi import mlab
from numba import jit

import numpy as np
from scipy.ndimage import gaussian_filter
from mayavi import mlab
from numba import jit

# ===================================
# Step 1: Create Smooth Terrain
# ===================================
grid_size = 5
resolution = 0.05  # Slightly coarser resolution for better performance
x = np.arange(0, grid_size, resolution)
y = np.arange(0, grid_size, resolution)
X, Y = np.meshgrid(x, y)

amplitude_variation = 0.3
frequency_variation_x = 2 * np.pi * (1 * np.random.rand(*X.shape))
frequency_variation_y = 2 * np.pi * (1 * np.random.rand(*Y.shape))
Z = amplitude_variation * np.sin(frequency_variation_x * X) * np.cos(frequency_variation_y * Y)

num_bumps = 2000
for _ in range(num_bumps):
    x_center = np.random.uniform(0, 0)
    y_center = np.random.uniform(0, 0)
    height = 0.025 * np.random.rand()
    width = 0.05 + 0.1 * np.random.rand()
    Z += height * np.exp(-((X - x_center)**2 + (Y - y_center)**2) / (2 * width**2))

Z = np.abs(Z)
Z = gaussian_filter(Z, sigma=20)

# ===================================
# Step: Custom Rover Drawing Function
# ===================================
def draw_custom_rover(center):
    """
    Draws a custom rover using Mayavi primitives.

    Parameters:
        center (list or np.ndarray): Center position of the rover [x, y, z].
    Returns:
        list: A list of Mayavi objects representing the rover (body and wheels).
    """
    # Body as a cylinder
    body = mlab.plot3d(
        [center[0], center[0]], 
        [center[1], center[1]], 
        [center[2], center[2] + 0.2],  # Height of the body
        tube_radius=0.1, color=(1, 0, 0)
    )
    return [body]


rover_center = [2.5, 2.5, 0.2]  # Centered on the terrain with height above Z.max()


# ===================================
# Step 2: Particle Initialization
# ===================================
particle_radius = 0.01
particle_diameter = 2 * particle_radius

positions = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        terrain_height = Z[i, j]
        num_particles = max(1, int(terrain_height / particle_diameter))  # Ensure at least one particle
        for k in range(num_particles):
            px, py = X[i, j], Y[i, j]
            pz = k * particle_diameter + particle_radius
            positions.append([px, py, pz])

positions = np.array(positions)

velocities = np.zeros_like(positions)
masses = np.ones(len(positions))
radii = np.full(len(positions), particle_radius)

rover_velocity = np.array([-0.3, 0.3])
rover_center = np.array(rover_center)
rover_size = np.array([0.4, 0.4, 0.1])
rover_velocity = np.array(rover_velocity)

@jit(nopython=True)
def rover_particle_interact(rover_center, rover_size, rover_velocity, particle_positions, particle_velocities):
    half_size = 0.5 * rover_size
    rmin = rover_center - half_size
    rmax = rover_center + half_size

    for i in range(len(particle_positions)):
        px, py, pz = particle_positions[i]
        bx, by, bz = rmin
        tx, ty, tz = rmax

        if (bx <= px <= tx and by <= py <= ty and bz <= pz <= tz):
            particle_velocities[i] += rover_velocity * 0.8


@jit(nopython=True)
def particle_collision(positions, velocities, masses, radii, restitution):
    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            diff = positions[j] - positions[i]
            dist_sq = np.sum(diff**2)
            min_dist = radii[i] + radii[j]
            if dist_sq < (min_dist ** 2) and dist_sq > 1e-12:
                dist = np.sqrt(dist_sq)
                normal = diff / dist
                rel_vel = velocities[j] - velocities[i]
                vel_along_normal = np.dot(rel_vel, normal)
                if vel_along_normal < 0:
                    m1, m2 = masses[i], masses[j]
                    j_impulse = -(1 + restitution) * vel_along_normal
                    j_impulse /= (1/m1 + 1/m2)
                    impulse = j_impulse * normal
                    velocities[int(i)] -= (impulse / m1)
                    velocities[int(j)] += (impulse / m2)
        return velocities

@jit(nopython=True)
def apply_layer_friction(positions, velocities, radii, friction_coefficient):
    """
    Apply friction between particles in adjacent layers.

    Parameters:
        positions (np.ndarray): Array of particle positions (N x 3).
        velocities (np.ndarray): Array of particle velocities (N x 3).
        radii (np.ndarray): Array of particle radii.
        friction_coefficient (float): Coefficient of friction between layers.
    """
    num_particles = len(positions)
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            # Check if particles are vertically aligned (same x, y)
            if np.abs(positions[i, 0] - positions[j, 0]) < radii[i] and \
               np.abs(positions[i, 1] - positions[j, 1]) < radii[i]:
                # Determine which particle is above and which is below
                if positions[i, 2] > positions[j, 2]:
                    upper_idx, lower_idx = i, j
                else:
                    upper_idx, lower_idx = j, i

                # Apply friction to the upper particle
                relative_velocity = velocities[upper_idx] - velocities[lower_idx]
                friction_force = -friction_coefficient * relative_velocity
                velocities[upper_idx] += friction_force

# ===================================
# Step 4: Visualization with Mayavi
# ===================================
mlab.figure(size=(800, 600), bgcolor=(0.9, 0.9, 1))

# Plot terrain as a surface
mlab.surf(X, Y, Z.T, colormap='terrain', warp_scale=1)

# Plot particles
particle_plot = mlab.points3d(
    positions[:, 0], positions[:, 1], positions[:, 2],
    scale_factor=particle_diameter,
    color=(0.8, 0.7, 0.6),
)


# Add axes and set view
mlab.axes(extent=[0, grid_size, 0, grid_size, 0, Z.max()])
mlab.view(azimuth=45, elevation=60, distance=8)

# ===================================
# Step 5: Animation with Friction
# ===================================
# ===================================
# Step 5: Animation with Friction
# ===================================
@mlab.animate(delay=10)
def anim():
    global positions, velocities
    dt = 0.02
    friction_coefficient = 0.3
    rover_velocity = [-0.3, 0.3]

    # Initialize rover visualization
    rover_objects = draw_custom_rover(rover_center)

    for _ in range(200):  # Number of simulation steps
        # Move the rover
        rover_center[0] += rover_velocity[0] * dt
        rover_center[1] += rover_velocity[1] * dt

        # Check collisions of rover with particles
        rover_particle_interact(rover_center, [0.4, 0.4, 0.1], rover_velocity, positions, velocities)

        # Handle particle-particle collisions
        velocities = particle_collision(positions, velocities, masses, radii, restitution=0.6)

        # Apply layer-based friction
        apply_layer_friction(positions, velocities, radii, friction_coefficient)

        # Update positions with boundary checks
        for i in range(len(positions)):
            grid_x = int(positions[i][0] / resolution)
            grid_y = int(positions[i][1] / resolution)
            if 0 <= grid_x < Z.shape[1] and 0 <= grid_y < Z.shape[0]:
                terrain_height = Z[grid_y][grid_x]
                if positions[i][2] < terrain_height:
                    positions[i][2] = terrain_height
                    velocities[i][2] *= -1

        # Update visualization (sampled for performance)
        particle_plot.mlab_source.set(
            x=positions[:, 0], y=positions[:, 1], z=positions[:, 2]
        )

        # Update custom rover position dynamically
        body = rover_objects[0]
        
        body.mlab_source.set(
            x=[rover_center[0], rover_center[0]],
            y=[rover_center[1], rover_center[1]],
            z=[rover_center[2], rover_center[2] + 0.2]
        )
        yield

anim()
mlab.show()



In [ ]:
def draw_custom_rover(center):
    """
    Draws a custom rover using Mayavi primitives.

    Parameters:
        center (list or np.ndarray): Center position of the rover [x, y, z].
    """
    # Body as a cylinder
    mlab.plot3d(
        [center[0], center[0]], 
        [center[1], center[1]], 
        [center[2], center[2] + 0.2],  # Height of the body
        tube_radius=0.1, color=(1, 0, 0)
    )
    
    # Wheels as spheres
    wheel_offsets = [[-0.15, -0.15], [-0.15, 0.15], [0.15, -0.15], [0.15, 0.15]]
    for offset in wheel_offsets:
        mlab.points3d(
            center[0] + offset[0],
            center[1] + offset[1],
            center[2],
            scale_factor=0.05,
            color=(0, 0, 1)
        )


In [ ]:
from scipy.ndimage import gaussian_filter
from mayavi import mlab
from numba import jit

import numpy as np
from scipy.ndimage import gaussian_filter
from mayavi import mlab
from numba import jit

global positions, velocities, rover_center 
# ===================================
# Step 1: Create Smooth Terrain
# ===================================
grid_size = 5
resolution = 0.05  # Slightly coarser resolution for better performance
x = np.arange(0, grid_size, resolution)
y = np.arange(0, grid_size, resolution)
X, Y = np.meshgrid(x, y)

amplitude_variation = 0.3
frequency_variation_x = 2 * np.pi * (1 * np.random.rand(*X.shape))
frequency_variation_y = 2 * np.pi * (1 * np.random.rand(*Y.shape))
Z = amplitude_variation * np.sin(frequency_variation_x * X) * np.cos(frequency_variation_y * Y)

num_bumps = 2000
for _ in range(num_bumps):
    x_center = np.random.uniform(0, 0)
    y_center = np.random.uniform(0, 0)
    height = 0.025 * np.random.rand()
    width = 0.05 + 0.1 * np.random.rand()
    Z += height * np.exp(-((X - x_center)**2 + (Y - y_center)**2) / (2 * width**2))

Z = np.abs(Z)
Z = gaussian_filter(Z, sigma=20)


# ===================================
# Step: Custom Rover Drawing Function
# ===================================
def draw_custom_rover(center):
    """
    Draws a custom rover using Mayavi primitives.

    Parameters:
        center (list or np.ndarray): Center position of the rover [x, y, z].
    Returns:
        list: A list of Mayavi objects representing the rover (body and wheels).
    """
    # Body as a cylinder
    body = mlab.plot3d(
        [center[0], center[0]], 
        [center[1], center[1]], 
        [center[2], center[2] + 0.2],  # Height of the body
        tube_radius=0.1, color=(1, 0, 0)
    )
    return [body]

rover_center = [2.5, 2.5, 0.2]  # Centered on the terrain with height above Z.max()
# Initialize rover parameters as NumPy arrays
rover_center = np.array([2.5, 2.5, 0.2])
rover_size = np.array([0.4, 0.4, 0.1])
rover_velocity = np.array([-0.3, 0.3])


# ===================================
# Step 2: Particle Initialization
# ===================================
particle_radius = 0.01
particle_diameter = 2 * particle_radius

positions = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        terrain_height = Z[i, j]
        num_particles = max(1, int(terrain_height / particle_diameter))  # Ensure at least one particle
        for k in range(num_particles):
            px, py = X[i, j], Y[i, j]
            pz = k * particle_diameter + particle_radius
            positions.append([px, py, pz])

positions = np.array(positions)

velocities = np.zeros_like(positions)
masses = np.ones(len(positions))
radii = np.full(len(positions), particle_radius)

@jit(nopython=True)
def rover_particle_interact(rover_center, rover_size, rover_velocity, particle_positions, particle_velocities):
    # Convert inputs to numpy arrays if they aren't already
    rover_size_array = rover_size#np.array(rover_size, dtype=np.float64)
    rover_velocity_array = rover_velocity#np.array(rover_velocity, dtype=np.float64)
    
    # Calculate half size
    half_size = 0.5 * rover_size_array
    rmin = rover_center - half_size
    rmax = rover_center + half_size

    for i in range(len(particle_positions)):
        px, py, pz = particle_positions[i]
        bx, by, bz = rmin
        tx, ty, tz = rmax

        if (bx <= px <= tx and by <= py <= ty and bz <= pz <= tz):
            particle_velocities[i][0] += rover_velocity_array[0] * 0.8
            particle_velocities[i][1] += rover_velocity_array[1] * 0.8

rover_size_np = np.array([0.4, 0.4, 0.1], dtype=np.float64)
rover_velocity_np = np.array(rover_velocity, dtype=np.float64)

@jit(nopython=True)
def particle_collision(positions, velocities, masses, radii, restitution):
    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            diff = positions[j] - positions[i]
            dist_sq = np.sum(diff**2)
            min_dist = radii[i] + radii[j]
            if dist_sq < (min_dist ** 2) and dist_sq > 1e-12:
                dist = np.sqrt(dist_sq)
                normal = diff / dist
                rel_vel = velocities[j] - velocities[i]
                vel_along_normal = np.dot(rel_vel, normal)
                if vel_along_normal < 0:
                    m1, m2 = masses[i], masses[j]
                    j_impulse = -(1 + restitution) * vel_along_normal
                    j_impulse /= (1/m1 + 1/m2)
                    impulse = j_impulse * normal
                    velocities[int(i)] -= (impulse / m1)
                    velocities[int(j)] += (impulse / m2)
    return velocities

@jit(nopython=True)
def apply_layer_friction(positions, velocities, radii, friction_coefficient):
    """
    Apply friction between particles in adjacent layers.

    Parameters:
        positions (np.ndarray): Array of particle positions (N x 3).
        velocities (np.ndarray): Array of particle velocities (N x 3).
        radii (np.ndarray): Array of particle radii.
        friction_coefficient (float): Coefficient of friction between layers.
    """
    num_particles = len(positions)
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            # Check if particles are vertically aligned (same x, y)
            if np.abs(positions[i, 0] - positions[j, 0]) < radii[i] and \
               np.abs(positions[i, 1] - positions[j, 1]) < radii[i]:
                # Determine which particle is above and which is below
                if positions[i, 2] > positions[j, 2]:
                    upper_idx, lower_idx = i, j
                else:
                    upper_idx, lower_idx = j, i

                # Apply friction to the upper particle
                relative_velocity = velocities[upper_idx] - velocities[lower_idx]
                friction_force = -friction_coefficient * relative_velocity
                velocities[upper_idx] += friction_force

# ===================================
# Step 4: Visualization with Mayavi
# ===================================
mlab.figure(size=(800, 600), bgcolor=(0.9, 0.9, 1))

# Plot terrain as a surface
mlab.surf(X, Y, Z.T, colormap='terrain', warp_scale=1)

# Plot particles
particle_plot = mlab.points3d(
    positions[:, 0], positions[:, 1], positions[:, 2],
    scale_factor=particle_diameter,
    color=(0.8, 0.7, 0.6),
)


# Add axes and set view
mlab.axes(extent=[0, grid_size, 0, grid_size, 0, Z.max()])
mlab.view(azimuth=45, elevation=60, distance=8)

# ===================================
# Step 5: Animation with Friction
# ===================================
@mlab.animate(delay=10)
def anim():
    global positions, velocities
    dt = 0.02
    friction_coefficient = 0.1  # Reduced friction for more movement

    # Initialize rover visualization
    rover_objects = draw_custom_rover(rover_center)

    for _ in range(200):  # Number of simulation steps
        # Move the rover
        rover_center[0] += rover_velocity[0] * dt
        rover_center[1] += rover_velocity[1] * dt

        # Check collisions of rover with particles
        rover_particle_interact(rover_center, rover_size_np, rover_velocity_np, positions, velocities)
        
        # Handle particle-particle collisions
        velocities = particle_collision(positions, velocities, masses, radii, restitution=0.6)

        # Apply layer-based friction
        apply_layer_friction(positions, velocities, radii, friction_coefficient)

        # Update positions with boundary checks
        for i in range(len(positions)):
            grid_x = int(positions[i][0] / resolution)
            grid_y = int(positions[i][1] / resolution)
            if 0 <= grid_x < Z.shape[1] and 0 <= grid_y < Z.shape[0]:
                terrain_height = Z[grid_y][grid_x]
                if positions[i][2] < terrain_height:
                    positions[i][2] = terrain_height
                    velocities[i][2] *= -1

        # Update particle visualization (sampled for performance)
        particle_plot.mlab_source.set(
            x=positions[:, 0],
            y=positions[:, 1],
            z=positions[:, 2]
        )

        # Update custom rover position dynamically
        body = rover_objects[0]
        body.mlab_source.set(
            x=[rover_center[0], rover_center[0]],
            y=[rover_center[1], rover_center[1]],
            z=[rover_center[2], rover_center[2] + 0.2]
        )

        yield

anim()
mlab.show()



TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C), dtype=class(float64))
 
There are 2 candidate implementations:
[1m      - Of which 2 did not match due to:
      Overload in function 'impl_np_array': File: numba\np\arrayobj.py: Line 5432.
        With argument(s): '(array(float64, 1d, C), dtype=class(float64))':[0m
[1m       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       [1m[1m[1mNo implementation of function Function(<intrinsic np_array>) found for signature:
        
        >>> np_array(array(float64, 1d, C), class(float64))
        
       There are 2 candidate implementations:
       [1m      - Of which 2 did not match due to:
             Intrinsic in function 'np_array': File: numba\np\arrayobj.py: Line 5406.
            